In [33]:
!pip install keras_preprocessing

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler


In [65]:
df = pd.read_csv("news_text.csv", on_bad_lines='skip',sep="\t")
df.head()

,news_id,title,abstract,category
0,N55528,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",lifestyle
1,N19639,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,health
2,N61837,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,news
3,N53526,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",health
4,N38324,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",health


In [66]:
df.shape

(51282, 4)

In [87]:
df.dropna()

def join(x):
  return ' '.join(str(x))

# print(df[df.columns[:-1]].apply(join,axis=1))
new_df=pd.DataFrame({"News":df[df.columns[:-1]].apply(join,axis=1),"Category":df[df.columns[-1]]})

new_df.head()

,News,Category
0,n e w s _ i d ...,lifestyle
1,n e w s _ i d ...,health
2,n e w s _ i d ...,news
3,n e w s _ i d ...,health
4,n e w s _ i d ...,health


In [88]:
max=0
for i in new_df['News']:
  if len(i.split())>max:
    max=len(i.split())
print(max)

142


In [90]:
special_characters = '"!@#$%^&*()-+?_|=,<>/"'

def tokenize(x):
    tokenizer = Tokenizer(
        filters=special_characters,
        lower=True,
        split=" "
    )
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

def pad(x, length=None):
    return pad_sequences(x, maxlen = 142, padding = 'post')

def preproc(x,length=None):
  seq,tok=tokenize(x.apply(lambda x:str(x)))
  padded_value=pad(seq,length)
  #reshaped_value=padded_value.reshape(*padded_value.shape,1)
  return padded_value,tok

X,data_tokenizer=preproc(new_df["News"])

category = new_df['Category'].unique()
dictionary = {}
i=1
for cate in category:
  dictionary[cate]=i
  i+=1

Y=new_df['Category'].apply(lambda x:dictionary[x])

print(X.shape)
print(Y.shape)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(51282, 142)
(51282,)
(41025, 142)
(41025,)
(10257, 142)
(10257,)


In [91]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [92]:
print(len(category))

17


In [93]:
# k_values = [i for i in range (1,31)]
# scores = []

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# for k in k_values:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     score = cross_val_score(knn, X,Y,cv=5)
#     scores.append(np.mean(score))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: ignored

In [96]:
# best_index = np.argmax(scores)
# best_k = k_values[best_index]

knn = KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train, y_train)



KNeighborsClassifier(n_neighbors=17)

In [97]:
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.34123037925319294
